In [93]:
import numpy as np
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, AveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import keras
from sklearn.utils import class_weight
import random

In [94]:
data = np.load("dermamnist.npz")
train_data = data["train_images"]
val_data = data["val_images"]
test_data = data["test_images"]
train_labels = data["train_labels"]
val_labels = data["val_labels"]
test_labels = data["test_labels"]

In [95]:
def get_counts(labels, class_num):
    counts = np.zeros(class_num)
    for label in labels:
        counts[label]+=1
    
    counts = [i/len(labels) for i in counts]

    return counts

train_counts = get_counts(train_labels, 7)

In [96]:
def shuffle_together(data, labels):
    p = np.random.permutation(len(data))
    return data[p], labels[p]

def oversample_class(data, labels, minority_class):

    all_indices = np.where(labels == minority_class)[0]
    data_to_add = np.zeros((len(all_indices), 28, 28, 3))
    print(len(all_indices))
    labels_to_add = np.zeros((len(all_indices), 1), dtype=int)
    print(np.shape(labels), np.shape(labels_to_add))

    for i in range(len(all_indices)):
        rnd_index = random.randint(0,len(all_indices)-1)
        data_to_add[i] = data[rnd_index].copy()
        labels_to_add[i] = labels[rnd_index].copy()

    data = np.append(data, data_to_add, axis=0)
    print(labels_to_add)
    labels = np.append(labels, labels_to_add, axis=0)

    return data, labels


# print(train_data)
print(get_counts(train_labels, 7))
print(np.shape(train_labels))
train_data, train_labels = oversample_class(train_data, train_labels, 2)
print(np.shape(train_labels))
train_data, train_labels = shuffle_together(train_data, train_labels)
print(get_counts(train_labels, 7))
# y = train_data[0].copy()
# print(y)
# print(train_data[0])

[0.03253888968174683, 0.051234479805908374, 0.10974739546168118, 0.01141715427429713, 0.11117453974596832, 0.6697588126159555, 0.0141287284144427]
(7007, 1)
769
(7007, 1) (769, 1)
[[5]
 [4]
 [2]
 [2]
 [5]
 [2]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [0]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [1]
 [5]
 [2]
 [4]
 [4]
 [5]
 [4]
 [5]
 [5]
 [5]
 [2]
 [1]
 [1]
 [5]
 [5]
 [2]
 [5]
 [1]
 [2]
 [5]
 [5]
 [4]
 [2]
 [4]
 [2]
 [5]
 [5]
 [5]
 [4]
 [5]
 [2]
 [5]
 [5]
 [6]
 [5]
 [2]
 [4]
 [5]
 [4]
 [0]
 [5]
 [5]
 [4]
 [6]
 [5]
 [4]
 [5]
 [5]
 [5]
 [2]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [2]
 [2]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [0]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [2]
 [2]
 [6]
 [5]
 [5]
 [5]
 [1]
 [5]
 [1]
 [5]
 [5]
 [2]
 [5]
 [5]
 [4]
 [5]
 [5]
 [6]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [2]
 [4]
 [5]
 [2]
 [0]
 [2]
 [5]
 [5]
 [5]
 [5]
 [5]
 [1]
 [4]
 [4]
 [5]
 [6]
 [2]
 [1]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [2]
 [1]
 [2]
 [5]
 [5]
 [5]
 [5]


In [51]:
def one_hot_encode(labels, class_num):
    one_hot_labels = np.zeros((np.shape(labels)[0], class_num), dtype=int)
    for i, label in enumerate(labels):
        one_hot_labels[i][int(label[0])] = 1
    
    return one_hot_labels

train_labels = one_hot_encode(train_labels, 7)
val_labels = one_hot_encode(val_labels, 7)
test_labels = one_hot_encode(test_labels, 7)

In [263]:
# class_num = 7
# class_weights = {}
# for i in range(class_num):
#     class_weights[i] = np.max(train_counts)/train_counts[i]

In [265]:
keras.regularizers.l2(0.02)
cnn_model = Sequential()
cnn_model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape= (28, 28, 3)))
cnn_model.add(MaxPooling2D(2, 2))
# cnn_model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape= (28, 28, 3)))
# cnn_model.add(MaxPooling2D(2, 2))
cnn_model.add(Dropout(.1))
cnn_model.add(Flatten())
cnn_model.add(Dense(20, activation='relu'))
cnn_model.add(Dense(7, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')

In [266]:
cnn_model.fit(train_data, train_labels, batch_size=32, epochs=10)
results = cnn_model.predict(val_data)
cnn_model.evaluate(val_data, val_labels)

Epoch 1/10
219/219 [==============================] - 5s 9ms/step - loss: 3.0576 - accuracy: 0.6441
Epoch 2/10
219/219 [==============================] - 2s 10ms/step - loss: 1.6702 - accuracy: 0.6698
Epoch 3/10
219/219 [==============================] - 2s 8ms/step - loss: 1.5205 - accuracy: 0.6698
Epoch 4/10
219/219 [==============================] - 2s 8ms/step - loss: 1.4051 - accuracy: 0.6698
Epoch 5/10
219/219 [==============================] - 2s 7ms/step - loss: 1.4915 - accuracy: 0.6673
Epoch 6/10
219/219 [==============================] - 2s 7ms/step - loss: 1.2615 - accuracy: 0.6698
Epoch 7/10
219/219 [==============================] - 2s 7ms/step - loss: 1.2210 - accuracy: 0.6698
Epoch 8/10
219/219 [==============================] - 2s 8ms/step - loss: 1.1936 - accuracy: 0.6698
Epoch 9/10
219/219 [==============================] - 2s 8ms/step - loss: 1.1751 - accuracy: 0.6698
Epoch 10/10
32/32 [==============================] - 0s 4ms/step - loss: 1.1600 - accuracy: 0.6690


[1.1599526405334473, 0.6689929962158203]

In [267]:
print(results[1])
print(results[2])
print(results[3])
print(results[4])
print(results[5])

[0.04463231 0.05623441 0.10444973 0.03916988 0.10504017 0.6139832
 0.03649038]
[0.04463231 0.05623441 0.10444973 0.03916988 0.10504017 0.6139832
 0.03649038]
[0.04463231 0.05623441 0.10444973 0.03916988 0.10504017 0.6139832
 0.03649038]
[0.04463231 0.05623441 0.10444973 0.03916988 0.10504017 0.6139832
 0.03649038]
[0.04463231 0.05623441 0.10444973 0.03916988 0.10504017 0.6139832
 0.03649038]
